In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('/detanet-md/')
from detanet_model.detanet_pbc import *
from detanet_model.metrics import *
from torch_geometric.loader import DataLoader
from e3nn import o3
import os
from torch_geometric.data import Data
import warnings
warnings.filterwarnings('ignore')
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
model=DetaNet(  num_features=128,
                 act='swish',
                 maxl=3,
                 num_block=3,
                 radial_type='trainable_bessel',
                 num_radial=32,
                 attention_head=8,
                 cutoff_lower=0.0,
                 cutoff_upper=5.0,
                 max_num_neighbors=120,
                 strategy="brute",
                 check_errors=True,
                 box_vecs=None,
                 dropout=0.0,
                 use_cutoff=False,
                 max_atomic_number=35,
                 atom_ref=None,
                 scale=1.0,
                 scalar_outsize=2,
                 irreps_out='2e',
                 summation=True,
                 norm=False,
                 out_type='2_tensor',
                 grad_type=None,
                 device=torch.device('cuda'))

<All keys matched successfully>

In [3]:
class Trainer:
    def __init__(self,model,train_loader,val_loader=None,loss_function=l2loss,device=torch.device('cuda'),
                 optimizer='Adam_amsgrad',lr=5e-4,weight_decay=0):
        self.opt_type=optimizer
        self.device=device
        self.model=model
        self.train_data=train_loader
        self.val_data=val_loader
        self.device=device
        self.opts={'AdamW':torch.optim.AdamW(self.model.parameters(),lr=lr,amsgrad=False,weight_decay=weight_decay),
              'AdamW_amsgrad':torch.optim.AdamW(self.model.parameters(),lr=lr,amsgrad=True,weight_decay=weight_decay),
              'Adam':torch.optim.Adam(self.model.parameters(),lr=lr,amsgrad=False,weight_decay=weight_decay),
              'Adam_amsgrad':torch.optim.Adam(self.model.parameters(),lr=lr,amsgrad=True,weight_decay=weight_decay),
              'Adadelta':torch.optim.Adadelta(self.model.parameters(),lr=lr,weight_decay=weight_decay),
              'RMSprop':torch.optim.RMSprop(self.model.parameters(),lr=lr,weight_decay=weight_decay),
              'SGD':torch.optim.SGD(self.model.parameters(),lr=lr,weight_decay=weight_decay)
        }
        self.optimizer=self.opts[self.opt_type]
        self.loss_function=loss_function
        self.step=-1
    def train(self,num_epoch,targ,stop_loss=1e-8,element=None,q9=None,loss_area=[100000,1e-8,1e8],
              val_per_train=10,view_data=False,print_per_epoch=10,data_scale=1):
        self.model.train()
        len_train=len(self.train_data)
        epoch=num_epoch
        for i in range(epoch):
            val_datas=iter(self.val_data)
            for j,batch in enumerate(self.train_data):
                self.step=self.step+1
                torch.cuda.empty_cache()
                self.optimizer.zero_grad()
                out = self.model(pos=torch.tensor(batch.pos,device=device,dtype=torch.float32), z=batch.z.to(self.device),
                                     batch=batch.batch.to(self.device), box=batch.box.to(self.device),
                                )
                target = batch[targ].to(self.device)*data_scale
                loss = self.loss_function(out.reshape(target.shape),target)
                if self.step<loss_area[0]:
                    loss.backward()
                elif self.step>loss_area[0] and loss_area[1]<loss.item()<loss_area[2]:
                    loss.backward()
                self.optimizer.step()
                if (self.step%val_per_train==0) and (self.val_data is not None):
                    val_batch = next(val_datas)
                    val_target=val_batch[targ].to(self.device).reshape(-1)*data_scale

                    val_out = self.model(pos=torch.tensor(val_batch.pos,device=device,dtype=torch.float32), z=val_batch.z.to(self.device),
                                             batch=val_batch.batch.to(self.device),
                                             box=val_batch.box.to(self.device),
                                        ).reshape(val_target.shape)
                    val_loss = self.loss_function(val_out, val_target).item()
                    val_mae=l1loss(val_out, val_target).item()
                    val_R2=R2(val_out,val_target).item()
                    if self.step % print_per_epoch==0:
                        print('Epoch[{}/{}],loss:{:.8f},val_loss:{:.8f},val_mae:{:.8f},val_R2:{:.8f}'
                              .format(self.step,num_epoch*len_train,loss.item(),val_loss,val_mae,val_R2))


                        if view_data:
                            print('valout:{:.8f},valtarget:{:.8f}'.format(val_out.flatten()[0].item()
                                                                                   , val_target.flatten()[0].item()))
                    assert (loss > stop_loss) or (val_loss > stop_loss),'Training and prediction Loss is less' \
                                                                        ' than cut-off Loss, so training stops'
                elif (self.step % print_per_epoch == 0) and (self.step%val_per_train!=0):
                    print('Epoch[{}/{}],loss:{:.8f}'.format(self.step,num_epoch*len_train, loss.item()))
                if self.step%100==0:
                    self.save_param(path+str(self.step)+'.pth')
    def load_state_and_optimizer(self,state_path=None,optimizer_path=None):
        if state_path is not None:
            state_dict=torch.load(state_path)
            self.model.load_state_dict(state_dict)
        if optimizer_path is not None:
            self.optimizer=torch.load(optimizer_path)

    def save_param(self,path):
        torch.save(self.model.state_dict(),path)

    def save_model(self,path):
        torch.save(self.model, path)

    def save_opt(self,path):
        torch.save(self.optimizer,path)

    def params(self):
        return self.model.state_dict()


    def load_state_and_optimizer(self,state_path=None,optimizer_path=None):
        if state_path is not None:
            state_dict=torch.load(state_path)
            self.model.load_state_dict(state_dict)
        if optimizer_path is not None:
            self.optimizer=torch.load(optimizer_path)

    def save_param(self,path):
        torch.save(self.model.state_dict(),path)

    def save_model(self,path):
        torch.save(self.model, path)

    def save_opt(self,path):
        torch.save(self.optimizer,path)

    def params(self):
        return self.model.state_dict()

In [4]:
datasets1=torch.load('/glycine.pt')
datasets=datasets1[:2001]
print(datasets[0])
print(len(datasets))

Data(pos=[160, 3], z=[160], atomization_energy=98700.34375, force=[160, 3], dipole=[1, 3], polarizability=[1, 3, 3], box=[1, 3, 3])
2001


In [5]:
train_datasets=[]
val_datasets1=[]
for i in range(len(datasets)):
    zerr=0
    targ='polarizability'
    data=datasets[i]
    data_=Data(pos=data.pos,z=data.z,box=data.box,targ=data[targ])
    if zerr==0:
        if i%(10*1)==0:
            val_datasets1.append(data_)
        else:
            train_datasets.append(data_)
    else:
        print('error',data.smile)
print(len(train_datasets))
print(len(val_datasets1))

1800
201


In [6]:
val_datasets=[]
test_datasets = []
for i in range(len(val_datasets1)):
    d = val_datasets1[i]
    if i % (2*1) == 0:
        val_datasets.append(d)
    else:
        test_datasets.append(d)
print(len(val_datasets))
print(len(test_datasets))

101
100


In [9]:
bathes=8
trainloader=DataLoader(train_datasets,batch_size=bathes,shuffle=True)
valloader=DataLoader(val_datasets,batch_size=bathes,shuffle=True)

In [10]:
device=torch.device('cuda')
dtype=torch.float32
model=model.to(dtype)
model=model.to(device)

In [11]:
path = '/polar/polar'

In [12]:
trainer=Trainer(model,train_loader=trainloader,val_loader=valloader,loss_function=l2loss,lr=1e-3,weight_decay=0,optimizer='AdamW')

In [13]:
trainer.train(num_epoch=15,stop_loss=0,targ='targ',val_per_train=50,view_data=False,print_per_epoch=10,loss_area=[100,1e-8,1000])

Epoch[0/3375],loss:265452.50000000,val_loss:62349.05468750,val_mae:171.31135559,val_R2:-11.69941998
Epoch[10/3375],loss:1096.52233887
Epoch[20/3375],loss:325.51763916
Epoch[30/3375],loss:225.44816589
Epoch[40/3375],loss:79.89916992
Epoch[50/3375],loss:61.14820862,val_loss:42.42278671,val_mae:4.90955210,val_R2:0.99157906
Epoch[60/3375],loss:40.85384369
Epoch[70/3375],loss:33.53926086
Epoch[80/3375],loss:27.94861794
Epoch[90/3375],loss:23.76916122
Epoch[100/3375],loss:15.26951599,val_loss:17.57437515,val_mae:3.18111873,val_R2:0.99649167
Epoch[110/3375],loss:27.23576355
Epoch[120/3375],loss:20.59933281
Epoch[130/3375],loss:17.47947311
Epoch[140/3375],loss:32.99149704
Epoch[150/3375],loss:28.42362404,val_loss:36.45561981,val_mae:4.93346643,val_R2:0.99289924
Epoch[160/3375],loss:21.38323402
Epoch[170/3375],loss:18.96881294
Epoch[180/3375],loss:16.80446053
Epoch[190/3375],loss:21.40451241
Epoch[200/3375],loss:11.36441803,val_loss:9.62647343,val_mae:2.25694323,val_R2:0.99812871
Epoch[210/3375

In [14]:
trainer.save_param('/polar/polar1.pth')

In [15]:
model.load_state_dict(torch.load('/polar/polar1.pth'))

<All keys matched successfully>

In [17]:
trainer=Trainer(model,train_loader=trainloader,val_loader=valloader,loss_function=l2loss,lr=4e-4,weight_decay=0,optimizer='AdamW')

In [18]:
trainer.train(num_epoch=15,stop_loss=0,targ='targ',val_per_train=50,view_data=False,print_per_epoch=10,loss_area=[100,1e-8,100])

Epoch[0/3375],loss:1.01805508,val_loss:1403.97949219,val_mae:25.27550507,val_R2:0.72408283
Epoch[10/3375],loss:176.45706177
Epoch[20/3375],loss:35.47467804
Epoch[30/3375],loss:16.89331818
Epoch[40/3375],loss:4.28965235
Epoch[50/3375],loss:5.16324186,val_loss:2.83650255,val_mae:1.30749154,val_R2:0.99946678
Epoch[60/3375],loss:2.59706712
Epoch[70/3375],loss:2.07428789
Epoch[80/3375],loss:2.25986362
Epoch[90/3375],loss:1.78833556
Epoch[100/3375],loss:2.45157599,val_loss:1.98208451,val_mae:1.10378134,val_R2:0.99961030
Epoch[110/3375],loss:1.24365056
Epoch[120/3375],loss:1.43284619
Epoch[130/3375],loss:1.65911543
Epoch[140/3375],loss:1.60874617
Epoch[150/3375],loss:1.32966042,val_loss:2.54918981,val_mae:1.21307194,val_R2:0.99950469
Epoch[160/3375],loss:1.65261972
Epoch[170/3375],loss:1.78911555
Epoch[180/3375],loss:1.69319737
Epoch[190/3375],loss:1.18939126
Epoch[200/3375],loss:2.15408707,val_loss:3.48116159,val_mae:1.27430475,val_R2:0.99929202
Epoch[210/3375],loss:2.08117270
Epoch[220/3375

Epoch[9070/12660],loss:1.28447139
Epoch[9080/12660],loss:2.78401947
Epoch[9090/12660],loss:11.24308968
Epoch[9100/12660],loss:4.69065714,val_loss:1.50130415,val_mae:0.97179931,val_R2:0.99998403
Epoch[9110/12660],loss:1.93920386
Epoch[9120/12660],loss:1.47550523
Epoch[9130/12660],loss:4.56333637
Epoch[9140/12660],loss:3.35296178
Epoch[9150/12660],loss:1.26651037,val_loss:2.89628196,val_mae:1.34665346,val_R2:0.99996883
Epoch[9160/12660],loss:1.53388894
Epoch[9170/12660],loss:1.60164750
Epoch[9180/12660],loss:1.80766070
Epoch[9190/12660],loss:1.04790449
Epoch[9200/12660],loss:1.47032106,val_loss:1.00298285,val_mae:0.79948598,val_R2:0.99998945
Epoch[9210/12660],loss:2.11201692
Epoch[9220/12660],loss:1.65052736
Epoch[9230/12660],loss:2.01954222
Epoch[9240/12660],loss:1.52886295
Epoch[9250/12660],loss:1.54962099,val_loss:1.17096055,val_mae:0.84229827,val_R2:0.99998772
Epoch[9260/12660],loss:1.01040804
Epoch[9270/12660],loss:1.01097095
Epoch[9280/12660],loss:1.11244357
Epoch[9290/12660],loss:

Epoch[10860/12660],loss:0.89649689
Epoch[10870/12660],loss:0.86914271
Epoch[10880/12660],loss:0.88166720
Epoch[10890/12660],loss:1.61944532
Epoch[10900/12660],loss:1.24311411,val_loss:2.44341493,val_mae:1.24343479,val_R2:0.99997407
Epoch[10910/12660],loss:0.74315608
Epoch[10920/12660],loss:1.02279735
Epoch[10930/12660],loss:1.90914786
Epoch[10940/12660],loss:0.93422276
Epoch[10950/12660],loss:1.22781813,val_loss:2.02100396,val_mae:1.13891876,val_R2:0.99997854
Epoch[10960/12660],loss:0.84184593
Epoch[10970/12660],loss:1.35804856
Epoch[10980/12660],loss:1.30076087
Epoch[10990/12660],loss:0.70176464
Epoch[11000/12660],loss:1.84477580,val_loss:1.40060496,val_mae:0.93228978,val_R2:0.99998510
Epoch[11010/12660],loss:0.79492176
Epoch[11020/12660],loss:0.99001598
Epoch[11030/12660],loss:0.76072735
Epoch[11040/12660],loss:1.46346581
Epoch[11050/12660],loss:1.08868134,val_loss:1.29943240,val_mae:0.90913010,val_R2:0.99998635
Epoch[11060/12660],loss:1.47980535
Epoch[11070/12660],loss:2.81180525
Ep

Epoch[12640/12660],loss:4.03494358
Epoch[12650/12660],loss:3.83189869,val_loss:1.64199352,val_mae:0.99669075,val_R2:0.99998230


In [19]:
trainer.save_param('/polar/polar2.pth')

In [20]:
model.load_state_dict(torch.load('/polar/polar2.pth'))

<All keys matched successfully>

In [22]:
trainer=Trainer(model,train_loader=trainloader,val_loader=valloader,loss_function=l2loss,lr=2e-4,weight_decay=0,optimizer='AdamW')

In [23]:
trainer.train(num_epoch=15,stop_loss=0,targ='targ',val_per_train=50,view_data=False,print_per_epoch=10,loss_area=[100,1e-8,50])

Epoch[0/3375],loss:1.63349640,val_loss:99.67549133,val_mae:7.51240730,val_R2:0.98081881
Epoch[10/3375],loss:9.31682682
Epoch[20/3375],loss:6.88072300
Epoch[30/3375],loss:4.41017246
Epoch[40/3375],loss:3.77076268
Epoch[50/3375],loss:1.46307111,val_loss:0.73303181,val_mae:0.59104472,val_R2:0.99985337
Epoch[60/3375],loss:2.51773572
Epoch[70/3375],loss:0.85293609
Epoch[80/3375],loss:0.47358280
Epoch[90/3375],loss:0.59234267
Epoch[100/3375],loss:0.60226798,val_loss:0.65447682,val_mae:0.57629722,val_R2:0.99987704
Epoch[110/3375],loss:0.26017836
Epoch[120/3375],loss:0.59257877
Epoch[130/3375],loss:0.47692817
Epoch[140/3375],loss:0.76634765
Epoch[150/3375],loss:0.84931833,val_loss:0.78947407,val_mae:0.59706765,val_R2:0.99985003
Epoch[160/3375],loss:1.04375613
Epoch[170/3375],loss:0.53508735
Epoch[180/3375],loss:0.65790129
Epoch[190/3375],loss:1.24411070
Epoch[200/3375],loss:0.66009688,val_loss:0.62412429,val_mae:0.59843028,val_R2:0.99987650
Epoch[210/3375],loss:0.54775685
Epoch[220/3375],loss:

Epoch[10860/12660],loss:0.32917812
Epoch[10870/12660],loss:0.32066423
Epoch[10880/12660],loss:0.30932581
Epoch[10890/12660],loss:0.37277102
Epoch[10900/12660],loss:0.40836993,val_loss:0.66694081,val_mae:0.65444863,val_R2:0.99999291
Epoch[10910/12660],loss:0.28624916
Epoch[10920/12660],loss:0.28745216
Epoch[10930/12660],loss:0.48019579
Epoch[10940/12660],loss:0.74419403
Epoch[10950/12660],loss:0.44792515,val_loss:0.51909477,val_mae:0.54873359,val_R2:0.99999452
Epoch[10960/12660],loss:0.32895055
Epoch[10970/12660],loss:0.47935575
Epoch[10980/12660],loss:0.51157337
Epoch[10990/12660],loss:0.26989833
Epoch[11000/12660],loss:0.30669063,val_loss:0.40098378,val_mae:0.47689763,val_R2:0.99999577
Epoch[11010/12660],loss:0.46722397
Epoch[11020/12660],loss:0.39517525
Epoch[11030/12660],loss:0.24806516
Epoch[11040/12660],loss:0.31089181
Epoch[11050/12660],loss:0.39742476,val_loss:0.33179602,val_mae:0.43134755,val_R2:0.99999648
Epoch[11060/12660],loss:0.40861553
Epoch[11070/12660],loss:0.39158452
Ep

Epoch[12640/12660],loss:0.33177006
Epoch[12650/12660],loss:0.43024850,val_loss:0.34741512,val_mae:0.43400630,val_R2:0.99999630


In [24]:
trainer.save_param('/polar/polar3.pth')

In [25]:
model.load_state_dict(torch.load('/polar/polar3.pth'))

<All keys matched successfully>

In [27]:
trainer=Trainer(model,train_loader=trainloader,val_loader=valloader,loss_function=l2loss,lr=1e-4,weight_decay=0,optimizer='AdamW')

In [28]:
trainer.train(num_epoch=15,stop_loss=0,targ='targ',val_per_train=50,view_data=False,print_per_epoch=10,loss_area=[100,1e-8,20])

Epoch[0/3375],loss:0.60360259,val_loss:4.87137127,val_mae:1.45039046,val_R2:0.99903595
Epoch[10/3375],loss:1.28141487
Epoch[20/3375],loss:1.01673853
Epoch[30/3375],loss:0.54160964
Epoch[40/3375],loss:0.19483350
Epoch[50/3375],loss:0.63677877,val_loss:0.67278230,val_mae:0.61164522,val_R2:0.99987274
Epoch[60/3375],loss:0.17732368
Epoch[70/3375],loss:0.44410226
Epoch[80/3375],loss:0.90080750
Epoch[90/3375],loss:0.85745877
Epoch[100/3375],loss:0.55629343,val_loss:1.13157833,val_mae:0.61125690,val_R2:0.99977070
Epoch[110/3375],loss:0.48437133
Epoch[120/3375],loss:0.65013725
Epoch[130/3375],loss:0.37494758
Epoch[140/3375],loss:0.64428133
Epoch[150/3375],loss:0.73674566,val_loss:0.90358335,val_mae:0.58458859,val_R2:0.99982405
Epoch[160/3375],loss:0.68749642
Epoch[170/3375],loss:0.19816184
Epoch[180/3375],loss:0.58543330
Epoch[190/3375],loss:0.57626927
Epoch[200/3375],loss:0.68718243,val_loss:0.67152029,val_mae:0.56515819,val_R2:0.99986672
Epoch[210/3375],loss:0.56181031
Epoch[220/3375],loss:0

In [29]:
trainer.save_param('/polar/polar4.pth')

In [30]:
model.load_state_dict(torch.load('/polar/polar4.pth'))

<All keys matched successfully>

In [32]:
trainer=Trainer(model,train_loader=trainloader,val_loader=valloader,loss_function=l2loss,lr=4e-5,weight_decay=0,optimizer='AdamW')

In [33]:
trainer.train(num_epoch=15,stop_loss=0,targ='targ',val_per_train=50,view_data=False,print_per_epoch=10,loss_area=[100,1e-8,10])

Epoch[0/3375],loss:0.51176220,val_loss:1.90431023,val_mae:1.13980484,val_R2:0.99963665
Epoch[10/3375],loss:0.83572787
Epoch[20/3375],loss:0.39396209
Epoch[30/3375],loss:0.44278434
Epoch[40/3375],loss:0.28612927
Epoch[50/3375],loss:0.33673757,val_loss:1.01160896,val_mae:0.52520174,val_R2:0.99979764
Epoch[60/3375],loss:0.38693440
Epoch[70/3375],loss:0.16571936
Epoch[80/3375],loss:0.30481794
Epoch[90/3375],loss:0.38641998
Epoch[100/3375],loss:0.34732184,val_loss:0.56281132,val_mae:0.50014669,val_R2:0.99988842
Epoch[110/3375],loss:0.44821399
Epoch[120/3375],loss:0.73903739
Epoch[130/3375],loss:0.36452293
Epoch[140/3375],loss:0.89410669
Epoch[150/3375],loss:1.24862933,val_loss:0.53586912,val_mae:0.45201826,val_R2:0.99989456
Epoch[160/3375],loss:0.20722021
Epoch[170/3375],loss:0.36409774
Epoch[180/3375],loss:0.43962035
Epoch[190/3375],loss:0.27653682
Epoch[200/3375],loss:0.65271598,val_loss:0.37305349,val_mae:0.38035670,val_R2:0.99992722
Epoch[210/3375],loss:0.87260592
Epoch[220/3375],loss:0

In [34]:
trainer.save_param('/polar/polar5.pth')

In [35]:
model.load_state_dict(torch.load('/polar/polar5.pth'))

<All keys matched successfully>

In [37]:
trainer=Trainer(model,train_loader=trainloader,val_loader=valloader,loss_function=l2loss,lr=2e-5,weight_decay=0,optimizer='AdamW')

In [38]:
trainer.train(num_epoch=15,stop_loss=0,targ='targ',val_per_train=50,view_data=False,print_per_epoch=10,loss_area=[100,1e-8,1])

Epoch[0/3375],loss:0.34660149,val_loss:0.30904987,val_mae:0.41326505,val_R2:0.99993932
Epoch[10/3375],loss:0.85283470
Epoch[20/3375],loss:0.42154020
Epoch[30/3375],loss:0.45375055
Epoch[40/3375],loss:0.45137617
Epoch[50/3375],loss:1.28455865,val_loss:0.41519773,val_mae:0.41667414,val_R2:0.99992031
Epoch[60/3375],loss:0.42955604
Epoch[70/3375],loss:0.15205650
Epoch[80/3375],loss:0.23993437
Epoch[90/3375],loss:0.34651223
Epoch[100/3375],loss:0.65652490,val_loss:0.47756937,val_mae:0.38531923,val_R2:0.99990469
Epoch[110/3375],loss:0.30629888
Epoch[120/3375],loss:0.25497586
Epoch[130/3375],loss:0.20829397
Epoch[140/3375],loss:0.47288683
Epoch[150/3375],loss:0.43080854,val_loss:0.32931107,val_mae:0.36892435,val_R2:0.99993610
Epoch[160/3375],loss:0.21253337
Epoch[170/3375],loss:0.23507711
Epoch[180/3375],loss:0.49699566
Epoch[190/3375],loss:0.18762285
Epoch[200/3375],loss:0.24304602,val_loss:0.20705362,val_mae:0.31327453,val_R2:0.99995828
Epoch[210/3375],loss:0.10566862
Epoch[220/3375],loss:0

In [39]:
trainer.save_param('/polar/polar6.pth')

In [40]:
model.load_state_dict(torch.load('/polar/polar6.pth'))

<All keys matched successfully>

In [42]:
trainer=Trainer(model,train_loader=trainloader,val_loader=valloader,loss_function=l2loss,lr=1e-5,weight_decay=0,optimizer='AdamW')

In [43]:
trainer.train(num_epoch=15,stop_loss=0,targ='targ',val_per_train=50,view_data=False,print_per_epoch=10,loss_area=[100,1e-8,1])

Epoch[0/3375],loss:0.48467112,val_loss:0.30513379,val_mae:0.35670227,val_R2:0.99994034
Epoch[10/3375],loss:0.60952318
Epoch[20/3375],loss:0.21930251
Epoch[30/3375],loss:0.32103437
Epoch[40/3375],loss:0.54291940
Epoch[50/3375],loss:0.58195257,val_loss:0.34154075,val_mae:0.38546896,val_R2:0.99993384
Epoch[60/3375],loss:0.42482001
Epoch[70/3375],loss:0.44148982
Epoch[80/3375],loss:0.18284763
Epoch[90/3375],loss:0.27047879
Epoch[100/3375],loss:0.19629191,val_loss:0.27408764,val_mae:0.34484732,val_R2:0.99994731
Epoch[110/3375],loss:0.33849770
Epoch[120/3375],loss:0.20218380
Epoch[130/3375],loss:1.24561059
Epoch[140/3375],loss:0.26321781
Epoch[150/3375],loss:0.29397827,val_loss:0.93473232,val_mae:0.53370935,val_R2:0.99981034
Epoch[160/3375],loss:0.25406376
Epoch[170/3375],loss:0.78742272
Epoch[180/3375],loss:0.30513543
Epoch[190/3375],loss:0.13503894
Epoch[200/3375],loss:0.18184824,val_loss:0.20043451,val_mae:0.29981667,val_R2:0.99995959
Epoch[210/3375],loss:0.71692455
Epoch[220/3375],loss:0

In [44]:
trainer.save_param('/polar/polar7.pth')